- RAG cmbines the capabilities of data retrieval and generation models to provide context-aware responses.
- User's query is passed to a search engine and the retrieved results are given to the model along with the query.
- The model then generates a response based on the query and retrieved information.

# Agentic RAG

- It extends traditional RAG systems by combining autonomous agents with dynamic knowledge retrieval.
- while traditional RAG systems use an LLM to answer queries based on retrieved data, agentic RAG enables intelligent control of both retrieval and generation processes, improving efficiency and accuracy.
- Traditional RAG systems face key limitations, such as relying on a single retrieval step and focusing on direct semantic similarity with the user's query, which may overlook relevant information.
- Agentic RAG addresses these issues by allowing the agent to autonomously formulate search queries, critique retrieved results and conduct multiple retrieval steps for a more tailored and comprehensive output.


In [1]:
from smolagents import CodeAgent, DuckDuckGoSearchTool, InferenceClientModel

# Initialize the search tool
search_tool = DuckDuckGoSearchTool()

# Initialize the model
model = InferenceClientModel()

agent = CodeAgent(
    model=model,
    tools=[search_tool],
)

# Example usage
response = agent.run(
    "Search for luxury superhero-themed party ideas, including decorations, entertainment, and catering."
)
print(response)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Search for luxury superhero-themed party ideas, including decorations, entertainment, and catering.             │
│                                                                                                                 │
╰─ InferenceClientModel - Qwen/Qwen2.5-Coder-32B-Instruct ────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Error in generating model output:
402 Client Error: Payment Required for url: 
https://router.huggingface.co/hf-inference/models/Qwen/Qwen2.5-Coder-32B-Instruct/v1/chat/completions (Request ID: 
Root=1-6849411a-315d6b41713f2d35040a0dd7;d29cf227-186c-4f89-b797-3b4e97e04479)

You have exceeded your monthly included credits for Inference Providers. Subscribe to PRO to get 20x more monthly 
included credits.

[Step 1: Duration 4.74 seconds]

AgentGenerationError: Error in generating model output:
402 Client Error: Payment Required for url: https://router.huggingface.co/hf-inference/models/Qwen/Qwen2.5-Coder-32B-Instruct/v1/chat/completions (Request ID: Root=1-6849411a-315d6b41713f2d35040a0dd7;d29cf227-186c-4f89-b797-3b4e97e04479)

You have exceeded your monthly included credits for Inference Providers. Subscribe to PRO to get 20x more monthly included credits.

- The agent follows this process:

  1. Analyzes the Request: Alfred’s agent identifies the key elements of the query—luxury superhero-themed party planning, with focus on decor, entertainment, and catering.
  2. Performs Retrieval: The agent leverages DuckDuckGo to search for the most relevant and up-to-date information, ensuring it aligns with Alfred’s refined preferences for a luxurious event.
  3. Synthesizes Information: After gathering the results, the agent processes them into a cohesive, actionable plan for Alfred, covering all aspects of the party.
  4. Stores for Future Reference: The agent stores the retrieved information for easy access when planning future events, optimizing efficiency in subsequent tasks.


# Custom knowledge Base Tool

- For specialized tasks, a custom knowledge base can be invaluable. Let’s create a tool that queries a vector database of technical documentation or specialized knowledge. Using semantic search, the agent can find the most relevant information for Alfred’s needs.

- A vector database stores numerical representations (embeddings) of text or other data, created by machine learning models. It enables semantic search by identifying similar meanings in high-dimensional space.

- This approach combines predefined knowledge with semantic search to provide context-aware solutions for event planning. With specialized knowledge access, Alfred can perfect every detail of the party.


In [3]:
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from smolagents import Tool
from langchain_community.retrievers import BM25Retriever
from smolagents import CodeAgent, InferenceClientModel

class PartyPlanningRetrievalTool(Tool):
    name="party_planning_retriever"
    description="Uses semantic search to retrieve relevant party planning ideas for Alfred's superhero-themed party at Wayne Manor. "
    inputs={
        "query":{
            "type":"string",
            "description":"The query to perform. This should be a query related to party planning or superhero themes."
        }
    }
    output_type="string"
    def __init__(self,docs,**kwargs):
        super().__init__(**kwargs)
        self.retriever=BM25Retriever.from_documents(
            docs,k=5
        )
    def forward(self,query:str)->str:
        assert isinstance(query,str),"Your search query must be a string"
        docs=self.retriever.invoke(
            query,
        )
        return "\nRetrieved ideas:\n"+"".join(
            [
                f"\n\n===== Idea {str(i)}=====\n"+doc.page_content
                for i, doc in enumerate(docs)
            ]
        )
party_ideas = [
    {"text": "A superhero-themed masquerade ball with luxury decor, including gold accents and velvet curtains.", "source": "Party Ideas 1"},
    {"text": "Hire a professional DJ who can play themed music for superheroes like Batman and Wonder Woman.", "source": "Entertainment Ideas"},
    {"text": "For catering, serve dishes named after superheroes, like 'The Hulk's Green Smoothie' and 'Iron Man's Power Steak.'", "source": "Catering Ideas"},
    {"text": "Decorate with iconic superhero logos and projections of Gotham and other superhero cities around the venue.", "source": "Decoration Ideas"},
    {"text": "Interactive experiences with VR where guests can engage in superhero simulations or compete in themed games.", "source": "Entertainment Ideas"}
]
source_docs=[
    Document(page_content=doc["text"],metadata={"source":doc["source"]})
    for doc in party_ideas
]
text_splitter=RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    add_start_index=True,
    strip_whitespace=True,
    separators=["\n\n","\n","."," ",""],
)
docs_processed=text_splitter.split_documents(source_docs)
party_planning_retriever=PartyPlanningRetrievalTool(docs_processed)
agent=CodeAgent(tools=[party_planning_retriever],model=InferenceClientModel())
response=agent.run(
    "Find ideas for a luxury superhero-themed party, including entertainment,catering and decoration options."
)
print(response)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Find ideas for a luxury superhero-themed party, including entertainment,catering and decoration options.        │
│                                                                                                                 │
╰─ InferenceClientModel - Qwen/Qwen2.5-Coder-32B-Instruct ────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Error in generating model output:
402 Client Error: Payment Required for url: 
https://router.huggingface.co/hf-inference/models/Qwen/Qwen2.5-Coder-32B-Instruct/v1/chat/completions (Request ID: 
Root=1-68494c58-4de89e5e43dde16a3dec00ec;869b79de-3d34-411d-9e8f-461768214e59)

You have exceeded your monthly included credits for Inference Providers. Subscribe to PRO to get 20x more monthly 
included credits.

[Step 1: Duration 0.64 seconds]

AgentGenerationError: Error in generating model output:
402 Client Error: Payment Required for url: https://router.huggingface.co/hf-inference/models/Qwen/Qwen2.5-Coder-32B-Instruct/v1/chat/completions (Request ID: Root=1-68494c58-4de89e5e43dde16a3dec00ec;869b79de-3d34-411d-9e8f-461768214e59)

You have exceeded your monthly included credits for Inference Providers. Subscribe to PRO to get 20x more monthly included credits.

# Enhanced retrieval capabilities

- When building agentic RAG systems, the agent can employ sophisticated strategies like:
  1. Query Reformulation: Instead of using the raw user query, the agent can craft optimized search terms that better match the target documents
  2. Multi-Step Retrieval: The agent can perform multiple searches, using initial results to inform subsequent queries
  3. Source Integration: Information can be combined from multiple sources like web search and local documentation
  4. Result Validation: Retrieved content can be analyzed for relevance and accuracy before being included in responses
